**PyCity Schools Analysis**

In [1]:
import pandas as pd

In [2]:
path1 = 'Resources/schools_complete.csv'
schools = pd.read_csv(path1, encoding='iso-8859-1', low_memory=False)
schools.head()

,School ID,name,type,size,budget
0,0,Huang High School,District,2917,1910635
1,1,Figueroa High School,District,2949,1884411
2,2,Shelton High School,Charter,1761,1056600
3,3,Hernandez High School,District,4635,3022020
4,4,Griffin High School,Charter,1468,917500


In [3]:
path2 = 'Resources/students_complete.csv'
students = pd.read_csv(path2, encoding='iso-8859-1', low_memory=False)
students.head()

,Student ID,name,gender,grade,school,reading_score,math_score
0,0,Paul Bradley,M,9th,Huang High School,66,79
1,1,Victor Smith,M,12th,Huang High School,94,61
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58
4,4,Bonnie Ray,F,9th,Huang High School,97,84


**District Summary**

In [4]:
total_schools = len(schools['School ID'].unique())
total_students = len(students['Student ID'].unique())
total_budget = schools['budget'].sum()

av_math_score = round((students['math_score'].mean()),2)
av_read_score = round((students['reading_score'].mean()),2)

math_pass_df = students.loc[students['math_score'] >= 70]
math_pass = round((len(math_pass_df) / total_students * 100),2)
read_pass_df = students.loc[students['reading_score'] >= 70]
read_pass = round((len(read_pass_df) / total_students * 100),2)
# Below overall passing rate = % of students who passed both math and reading, later it's just an average
ov_pass_df = pd.merge(math_pass_df, read_pass_df, on='Student ID')
ov_pass = round((len(ov_pass_df) / total_students * 100),2)
distr_df = pd.DataFrame({'Total Schools': [total_schools], 'Total Students': [total_students], 'Total Budget':
                         [total_budget], 'Average Math Score': [av_math_score], 'Average Reading Score':
                         [av_read_score], '% Passing Math': [math_pass], '% Passing Reading': [read_pass],
                         '% Overall Passing Rate': [ov_pass]})
distr_df['Total Budget'] = distr_df['Total Budget'].map("$ {:,.2f}".format)
distr_df_answ = distr_df[['Total Schools', 'Total Students', 'Total Budget', 'Average Math Score', 'Average Reading Score',
                         '% Passing Math', '% Passing Reading', '% Overall Passing Rate']]
distr_df_answ

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
0,15,39170,"$ 24,649,428.00",78.99,81.88,74.98,85.81,65.17


**School Summary**

In [5]:
# Form first 5 answer columns
school_sum = schools[['name', 'type', 'size', 'budget']]
school_sum = school_sum.assign(Per_Student_Budget = (school_sum['budget'] / school_sum['size']))
school_sum.columns = ['School name', 'School Type', 'Total Students', 'Total School Budget', 'Per Student Budget']

# Form average reading and math scores
students_extr = students[['school', 'reading_score', 'math_score']]
stud_sum = students_extr.groupby(['school'])
stud_sum_df = pd.DataFrame(stud_sum.mean())
stud_sum_df.reset_index(inplace=True)
stud_sum_df.columns = ['School name', 'Average Reading Score', 'Average Math Score']

# Form supplemental columns to calculate % Passing math
students_extr_math = students[['school', 'math_score']]
math_all_df = students_extr_math.loc[students_extr_math['math_score'] >= 70]
math_sum = math_all_df.groupby(['school'])
math_sum_df = pd.DataFrame(math_sum.count())
math_sum_df.reset_index(inplace=True)
math_sum_df.columns = ['School name', 'Passed Math Count']

# Form supplemental columns to calculate % Passing reading
students_extr_read = students[['school', 'reading_score']]
read_all_df = students_extr_read.loc[students_extr_read['reading_score'] >= 70]
read_sum = read_all_df.groupby(['school'])
read_sum_df = pd.DataFrame(read_sum.count())
read_sum_df.reset_index(inplace=True)
read_sum_df.columns = ['School name', 'Passed Read Count']

# Merge everything
new_sum1 = pd.merge(school_sum, stud_sum_df, on='School name')
new_sum2 = pd.merge(new_sum1, math_sum_df, on='School name')
new_sum = pd.merge(new_sum2, read_sum_df, on='School name')
# Calculate all 3 % Passing
new_sum['% Passing Math'] = round((new_sum['Passed Math Count'] / new_sum['Total Students'] * 100),2)
new_sum['% Passing Reading'] = round((new_sum['Passed Read Count'] / new_sum['Total Students'] * 100),2)
new_sum['% Overall Passing Rate'] = (new_sum['% Passing Math'] + new_sum['% Passing Reading'])/2
# Make the answer beautiful
distr_df_new = new_sum[['School name', 'School Type', 'Total Students', 'Total School Budget', 'Per Student Budget',
                     'Average Reading Score', 'Average Math Score','% Passing Reading', '% Passing Math',
                     '% Overall Passing Rate']]
distr_answ = distr_df_new.set_index('School name')
distr_answ['Total School Budget'] = distr_answ['Total School Budget'].map("$ {:,.2f}".format)
distr_answ['Per Student Budget'] = distr_answ['Per Student Budget'].map("$ {:,.2f}".format)
distr_answ

,School Type,Total Students,Total School Budget,Per Student Budget,Average Reading Score,Average Math Score,% Passing Reading,% Passing Math,% Overall Passing Rate
School name,,,,,,,,,
Huang High School,District,2917,"$ 1,910,635.00",$ 655.00,81.182722,76.629414,81.32,65.68,73.500
Figueroa High School,District,2949,"$ 1,884,411.00",$ 639.00,81.158020,76.711767,80.74,65.99,73.365
Shelton High School,Charter,1761,"$ 1,056,600.00",$ 600.00,83.725724,83.359455,95.85,93.87,94.860
Hernandez High School,District,4635,"$ 3,022,020.00",$ 652.00,80.934412,77.289752,80.86,66.75,73.805
Griffin High School,Charter,1468,"$ 917,500.00",$ 625.00,83.816757,83.351499,97.14,93.39,95.265
Wilson High School,Charter,2283,"$ 1,319,574.00",$ 578.00,83.989488,83.274201,96.54,93.87,95.205
Cabrera High School,Charter,1858,"$ 1,081,356.00",$ 582.00,83.975780,83.061895,97.04,94.13,95.585
Bailey High School,District,4976,"$ 3,124,928.00",$ 628.00,81.033963,77.048432,81.93,66.68,74.305
Holden High School,Charter,427,"$ 248,087.00",$ 581.00,83.814988,83.803279,96.25,92.51,94.380


**Top Performing Schools (By Passing Rate)**

In [6]:
top_schools = distr_df_new.sort_values('% Overall Passing Rate', ascending=False)
top = top_schools.set_index('School name')
top['Total School Budget'] = top['Total School Budget'].map("$ {:,.2f}".format)
top['Per Student Budget'] = top['Per Student Budget'].map("$ {:,.2f}".format)
top5 = top.head(5)
top5

,School Type,Total Students,Total School Budget,Per Student Budget,Average Reading Score,Average Math Score,% Passing Reading,% Passing Math,% Overall Passing Rate
School name,,,,,,,,,
Cabrera High School,Charter,1858,"$ 1,081,356.00",$ 582.00,83.975780,83.061895,97.04,94.13,95.585
Thomas High School,Charter,1635,"$ 1,043,130.00",$ 638.00,83.848930,83.418349,97.31,93.27,95.290
Pena High School,Charter,962,"$ 585,858.00",$ 609.00,84.044699,83.839917,95.95,94.59,95.270
Griffin High School,Charter,1468,"$ 917,500.00",$ 625.00,83.816757,83.351499,97.14,93.39,95.265
Wilson High School,Charter,2283,"$ 1,319,574.00",$ 578.00,83.989488,83.274201,96.54,93.87,95.205


**Top Bottom Schools (By Passing Rate)**

In [7]:
bottom_schools = distr_df_new.sort_values('% Overall Passing Rate', ascending=True)
bottom = bottom_schools.set_index('School name')
bottom['Total School Budget'] = bottom['Total School Budget'].map("$ {:,.2f}".format)
bottom['Per Student Budget'] = bottom['Per Student Budget'].map("$ {:,.2f}".format)
bottom5 = bottom.head(5)
bottom5

,School Type,Total Students,Total School Budget,Per Student Budget,Average Reading Score,Average Math Score,% Passing Reading,% Passing Math,% Overall Passing Rate
School name,,,,,,,,,
Rodriguez High School,District,3999,"$ 2,547,363.00",$ 637.00,80.744686,76.842711,80.22,66.37,73.295
Figueroa High School,District,2949,"$ 1,884,411.00",$ 639.00,81.158020,76.711767,80.74,65.99,73.365
Huang High School,District,2917,"$ 1,910,635.00",$ 655.00,81.182722,76.629414,81.32,65.68,73.500
Johnson High School,District,4761,"$ 3,094,650.00",$ 650.00,80.966394,77.072464,81.22,66.06,73.640
Hernandez High School,District,4635,"$ 3,022,020.00",$ 652.00,80.934412,77.289752,80.86,66.75,73.805


**Math Scores by Grade**

In [8]:
grade_both = students.groupby(['school','grade'])
grade_both_df = pd.DataFrame(grade_both.mean())
grade_both_df.drop('Student ID', axis=1, inplace=True)
grade_both_df.reset_index(inplace=True)
piv_math = grade_both_df.pivot(index='school', columns='grade', values='math_score') #pivot function,very convenient
math_grade = piv_math[['9th', '10th', '11th', '12th']]
math_grade

grade,9th,10th,11th,12th
school,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248


**Reading Scores by Grade**

In [9]:
piv_read = grade_both_df.pivot(index='school', columns='grade', values='reading_score')
read_grade = piv_read[['9th', '10th', '11th', '12th']]
read_grade

grade,9th,10th,11th,12th
school,,,,
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564


**Scores by School Spending**

In [10]:
print(distr_answ['Per Student Budget'].min())
print(distr_answ['Per Student Budget'].max())

$ 578.00
$ 655.00


In [11]:
bins = [0, 585, 610, 635, 660]
labels_budg = ['<$ 585', '$ 585-610', '$ 610-635', '$ 635-660']
# It's important to use .copy(), so the initial df won't be changed
budg_df = distr_df_new.copy()
budg_df['Spending Ranges (Per Student)'] = pd.cut(budg_df["Per Student Budget"],bins,labels=labels_budg)
budg_df.drop('Per Student Budget', axis=1, inplace=True)
budg_df.drop('Total School Budget', axis=1, inplace=True)
budg_df.drop('Total Students', axis=1, inplace=True)
budg_grouped = budg_df.groupby(['Spending Ranges (Per Student)'])
budg_grouped_df = pd.DataFrame(budg_grouped.mean())
budg_grouped_df

,Average Reading Score,Average Math Score,% Passing Reading,% Passing Math,% Overall Passing Rate
Spending Ranges (Per Student),,,,,
<$ 585,83.933814,83.455399,96.610000,93.460000,95.035000
$ 585-610,83.885211,83.599686,95.900000,94.230000,95.065000
$ 610-635,82.425360,80.199966,89.535000,80.035000,84.785000
$ 635-660,81.368774,77.866721,82.995714,70.347143,76.671429


**Scores by School Size**

In [12]:
print(schools['size'].max())
print(schools['size'].min())

4976
427


In [13]:
bins = [0, 1000, 3000, 5000]
labels_size = ['Small (<1000)', 'Medium (1000-3000)', 'Large (>3000)']
size_df = distr_df_new.copy()
size_df['School Size'] = pd.cut(size_df['Total Students'],bins,labels=labels_size)
size_df.drop('Per Student Budget', axis=1, inplace=True)
size_df.drop('Total School Budget', axis=1, inplace=True)
size_df.drop('Total Students', axis=1, inplace=True)
size_grouped = size_df.groupby(['School Size'])
size_grouped_df = pd.DataFrame(size_grouped.mean())
size_grouped_df

,Average Reading Score,Average Math Score,% Passing Reading,% Passing Math,% Overall Passing Rate
School Size,,,,,
Small (<1000),83.929843,83.821598,96.100000,93.550000,94.825000
Medium (1000-3000),82.933187,81.176821,91.316667,84.648889,87.982778
Large (>3000),80.919864,77.063340,81.057500,66.465000,73.761250


**Scores by School Type**

In [15]:
school_type = distr_df_new.copy()
type_grouped = school_type.groupby('School Type')
type_grouped_df = pd.DataFrame(type_grouped.mean())
type_grouped_df.drop('Total School Budget', axis=1, inplace=True)
type_grouped_df.drop('Total Students', axis=1, inplace=True)
type_grouped_df['Per Student Budget'] = type_grouped_df['Per Student Budget'].map("$ {:,.2f}".format)
type_grouped_df

,Per Student Budget,Average Reading Score,Average Math Score,% Passing Reading,% Passing Math,% Overall Passing Rate
School Type,,,,,,
Charter,$ 599.50,83.896421,83.473852,96.586250,93.620000,95.103125
District,$ 643.57,80.966636,76.956733,80.798571,66.548571,73.673571


**PySchools Trend Analysis**

1. Charter school students are significantly more successful that students from District schools.
2. Meanwhile per student budget in average Charter school is smaller, than in District one, so there is no direct relation between money and quality of education, or probably there is an inefficient spending of a budget.
3. School size alse determines the scores - smaller schools have better passing rates, than large ones.